In this third notebook, we will go over the basics of submitting jobs via the SDK, either to a Ray cluster or directly to MCAD.

In [1]:
# Import pieces from codeflare-sdk
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration
from codeflare_sdk.cluster.auth import TokenAuthentication

In [ ]:
# Create authentication object for user permissions
# IF unused, SDK will automatically check for default kubeconfig, then in-cluster config
# KubeConfigFileAuthentication can also be used to specify kubeconfig path manually
auth = TokenAuthentication(
    token = "XXXXX",
    server = "XXXXX",
    skip_tls=False
)
auth.login()

Let's start by running through the same cluster setup as before:

In [2]:
# Create and configure our cluster object (and appwrapper)
cluster = Cluster(ClusterConfiguration(
    name='jobtest',
    namespace='default',
    num_workers=2,
    min_cpus=1,
    max_cpus=1,
    min_memory=4,
    max_memory=4,
    num_gpus=0,
    instascale=False
))

Written to: jobtest.yaml


In [3]:
# Bring up the cluster
cluster.up()
cluster.wait_ready()

Waiting for requested resources to be set up...
Requested cluster up and running!


In [4]:
cluster.details()

                  🚀 CodeFlare Cluster Details 🚀                  
                                                                   
 ╭───────────────────────────────────────────────────────────────╮ 
 │   Name                                                        │ 
 │   jobtest                                        Active ✅    │ 
 │                                                               │ 
 │   URI: ray://jobtest-head-svc.default.svc:10001               │ 
 │                                                               │ 
 │   ]8;id=366603;http://ray-dashboard-jobtest-default.apps.meyceoz-07122023.psap.aws.rhperfscale.org\Dashboard🔗]8;;\                                                 │ 
 │                                                               │ 
 │                       Cluster Resources                       │ 
 │   ╭── Workers ──╮  ╭───────── Worker specs(each) ─────────╮   │ 
 │   │  # Workers  │  │  Memory      CPU         GPU         │   │ 
 │   │             │  │                                      │   │ 
 │   │  2          │  │  4~4         1           0           │   │ 
 │   │             │  │                                      │   │ 
 │   ╰─────────────╯  ╰──────────────────────────────────────╯   │ 
 ╰───────────────────────────────────────────────────────────────╯

RayCluster(name='jobtest', status=<RayClusterStatus.READY: 'ready'>, workers=2, worker_mem_min=4, worker_mem_max=4, worker_cpu=1, worker_gpu=0, namespace='default', dashboard='http://ray-dashboard-jobtest-default.apps.meyceoz-07122023.psap.aws.rhperfscale.org')

This time, however, we are going to use the CodeFlare SDK to submit batch jobs via TorchX, either to the Ray cluster we have just brought up, or directly to MCAD.

In [5]:
from codeflare_sdk.job.jobs import DDPJobDefinition

First, let's begin by submitting to Ray, training a basic NN on the MNIST dataset:

In [6]:
jobdef = DDPJobDefinition(
    name="mnisttest",
    script="mnist.py",
    scheduler_args={"requirements": "requirements.txt"}
)
job = jobdef.submit(cluster)

The Ray scheduler does not support port mapping.


Now we can take a look at the status of our submitted job, as well as retrieve the full logs:

In [9]:
job.status()

AppStatus:
  msg: !!python/object/apply:ray.dashboard.modules.job.common.JobStatus
  - RUNNING
  num_restarts: -1
  roles:
  - replicas:
    - hostname: <NONE>
      id: 0
      role: ray
      state: !!python/object/apply:torchx.specs.api.AppState
      - 3
      structured_error_msg: <NONE>
    role: ray
  state: RUNNING (3)
  structured_error_msg: <NONE>
  ui_url: null

In [10]:
job.logs()

'[RayActor(name=\'mnist\', command=[\'bash\', \'-c\', "torchrun --rdzv_backend static --rdzv_endpoint $TORCHX_RANK0_HOST:49782 --rdzv_id \'mnisttest-bc7wx5t7hd0lcc\' --nnodes 2 --nproc_per_node 1 --node_rank \'0\' --tee 3 --role \'\' mnist.py"], env={\'TORCHX_TRACKING_EXPERIMENT_NAME\': \'default-experiment\', \'LOGLEVEL\': \'WARNING\', \'TORCHX_JOB_ID\': \'ray://torchx/mnisttest-bc7wx5t7hd0lcc\'}, num_cpus=1, num_gpus=0, min_replicas=2), RayActor(name=\'mnist\', command=[\'bash\', \'-c\', "torchrun --rdzv_backend static --rdzv_endpoint $TORCHX_RANK0_HOST:49782 --rdzv_id \'mnisttest-bc7wx5t7hd0lcc\' --nnodes 2 --nproc_per_node 1 --node_rank \'1\' --tee 3 --role \'\' mnist.py"], env={\'TORCHX_TRACKING_EXPERIMENT_NAME\': \'default-experiment\', \'LOGLEVEL\': \'WARNING\', \'TORCHX_JOB_ID\': \'ray://torchx/mnisttest-bc7wx5t7hd0lcc\'}, num_cpus=1, num_gpus=0, min_replicas=2)]\n2023-08-09 08:05:59,434\tINFO worker.py:1334 -- Using address 10.129.0.64:6379 set in the environment variable RAY_

You can also view organized logs, status, and other information directly through the Ray cluster's dashboard:

In [11]:
cluster.cluster_dashboard_uri()

'http://ray-dashboard-jobtest-default.apps.meyceoz-07122023.psap.aws.rhperfscale.org'

Once complete, we can bring our Ray cluster down and clean up:

In [12]:
job.status()

AppStatus:
  msg: !!python/object/apply:ray.dashboard.modules.job.common.JobStatus
  - SUCCEEDED
  num_restarts: -1
  roles:
  - replicas:
    - hostname: <NONE>
      id: 0
      role: ray
      state: !!python/object/apply:torchx.specs.api.AppState
      - 4
      structured_error_msg: <NONE>
    role: ray
  state: SUCCEEDED (4)
  structured_error_msg: <NONE>
  ui_url: null

In [13]:
cluster.down()

Now, an alternative option for job submission is to submit directly to MCAD, which will schedule pods to run the job with requested resources:

In [14]:
jobdef = DDPJobDefinition(
    name="mnistjob",
    script="mnist.py",
    scheduler_args={"namespace": "default"},
    j="1x1",
    gpu=0,
    cpu=1,
    memMB=8000,
    image="quay.io/project-codeflare/mnist-job-test:v0.0.1"
)
job = jobdef.submit()

Once again, we can look at job status and logs:

In [15]:
job.status()

/opt/homebrew/lib/python3.8/site-packages/torchx/schedulers/kubernetes_mcad_scheduler.py:1105: UserWarning: Warning - MCAD does not report individual replica statuses, but overall task status. Replica id  may not match status
  warnings.warn(msg)


AppStatus:
  msg: <NONE>
  num_restarts: -1
  roles:
  - replicas:
    - hostname: ''
      id: 0
      role: mnist
      state: !!python/object/apply:torchx.specs.api.AppState
      - 2
      structured_error_msg: <NONE>
    role: mnist
  state: RUNNING (3)
  structured_error_msg: <NONE>
  ui_url: null

In [20]:
job.logs()

'2023-08-09T15:10:09.515811959Z [0]:GPU available: False, used: False\n2023-08-09T15:10:09.515811959Z [0]:TPU available: False, using: 0 TPU cores\n2023-08-09T15:10:09.515811959Z [0]:IPU available: False, using: 0 IPUs\n2023-08-09T15:10:09.615986853Z [0]:\n2023-08-09T15:10:09.615986853Z [0]:  0%|          | 0/9912422 [00:00<?, ?it/s]\n2023-08-09T15:10:09.615986853Z [0]:100%|██████████| 9912422/9912422 [00:00<00:00, 298100016.81it/s]\n2023-08-09T15:10:09.916463153Z [0]:\n2023-08-09T15:10:09.916463153Z [0]:  0%|          | 0/28881 [00:00<?, ?it/s]\n2023-08-09T15:10:09.916463153Z [0]:100%|██████████| 28881/28881 [00:00<00:00, 219846994.24it/s]\n2023-08-09T15:10:09.916463153Z [0]:\n2023-08-09T15:10:09.916463153Z [0]:  0%|          | 0/1648877 [00:00<?, ?it/s]\n2023-08-09T15:10:09.916463153Z [0]:100%|██████████| 1648877/1648877 [00:00<00:00, 111410068.25it/s]\n2023-08-09T15:10:10.016659353Z [0]:\n2023-08-09T15:10:10.016659353Z [0]:  0%|          | 0/4542 [00:00<?, ?it/s]\n2023-08-09T15:10:1

This time, once the pods complete, we can clean them up alongside any other associated resources. The following command can also be used to delete jobs early for both Ray and MCAD submission:

In [21]:
job.cancel()

In [ ]:
auth.logout()